<a href="https://colab.research.google.com/github/xinyueli2896/raptranscription/blob/main/loader_clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [90]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
!pip install eyed3

In [ ]:
import os
import eyed3
import csv
import re
import json

In [ ]:
# @title clean
def remove_duplicates_and_corrupts(dir):
    for filename in os.listdir(dir):
        file_path = os.path.join(dir, filename)
        try: #remove corrupted file
            if eyed3.load(file_path) is None:
                # File can't be loaded, delete it
                os.remove(file_path)
                print(f"Deleted corrupt file: {filename}")
        except Exception as e:
            # Handle any other exception
            print(f"Error with file {filename}: {e}")

        name = filename.split('.')[0]
        if name.endswith(')'): #remove duplicates
           os.remove(os.path.join(dir,filename))
           print(f"Deleted duplicated file: {filename}")
def append_rows_to_csv(file_path, new_rows):
    with open(file_path, 'a', newline='') as file:
        writer = csv.writer(file)
        # Check if new_rows is a list of lists
        if not all(isinstance(row, list) for row in new_rows):
            # If new_rows is just a single list, wrap it in another list
            new_rows = [new_rows]
        writer.writerows(new_rows)
        print(new_rows)


remove_duplicates_and_corrupts("/content/drive/MyDrive/data/dataset_cleaned/train")
remove_duplicates_and_corrupts("/content/drive/MyDrive/data/dataset2_cleaned/test")

# remove the ones like 'artist: '
original_csv = '/content/drive/MyDrive/data/dataset2/metadata.csv' # your path to the csv
new_csv = '/content/drive/MyDrive/data/dataset2/metadata2.0.csv' # your path to new csv
append_rows_to_csv(new_csv, ['file_name','lyrics'])
with open(original_csv) as file:
    content = file.readlines()

dir = "/content/drive/MyDrive/data/dataset2/train"
for filename in os.listdir(dir):
    index = filename.split('.')[0].split('_')[1]
    if content[int(index)+1].split(',',1)[1].__contains__('：'):
      #delete meaningless chunk file
      os.remove(os.path.join(dir,filename))
      print(f"{filename} deleted")
    else:
      #write the meaningful row into new csv
      # print([content[int(index)+1].split(',')[0],content[int(index)+1].split(',')[1]])
      row_to_write = content[int(index)+1].strip().split(',')

      # Ensure the newline character is removed from the end of the second element
      row_to_write = [row_to_write[0], row_to_write[1].replace('\n', '')]

      # Write the split elements into the CSV file
      append_rows_to_csv(new_csv, [row_to_write])

dir = "/content/drive/MyDrive/data/dataset2/test"
for filename in os.listdir(dir):
    index = filename.split('.')[0].split('_')[1]
    if content[int(index)+1].split(',',1)[1].__contains__('：'):
      #delete meaningless chunk file
      os.remove(os.path.join(dir,filename))
      print(f"{filename} deleted")
    else:
      #write the meaningful row into new csv
      # print([content[int(index)+1].split(',')[0],content[int(index)+1].split(',')[1]])
      row_to_write = content[int(index)+1].strip().split(',')

      # Ensure the newline character is removed from the end of the second element
      row_to_write = [row_to_write[0], row_to_write[1].replace('\n', '')]

      # Write the split elements into the CSV file
      append_rows_to_csv(new_csv, [row_to_write])


In [ ]:
# @title
with open('/content/metadata7.csv') as file:
    content1 = file.readlines()

In [6]:
dir_csv = "/content/drive/MyDrive/data/dataset_cleaned/metadata.csv"

In [55]:
import re

def extract_and_clean_words(sentence):
    # Find words containing an asterisk
    words = sentence.split()
    # Filter words that contain an asterisk
    words_with_asterisk = [word for word in words if '*' in word]
    # Clean each word to remove non-alphabetic and non-asterisk characters
    cleaned_words = [''.join(re.findall(r'[A-Za-z*\'-]', word)) for word in words_with_asterisk]
    return cleaned_words

# Example usage:
sentence = "Here's a sentence n***a with ""some*words, including*2asterisks, and others without."

cleaned_words = extract_and_clean_words(sentence)

print(cleaned_words)
# Output: ['some*words', 'including*asterisks']


['n***a', 'some*words', 'including*asterisks']


In [ ]:
# @title unmask
# Opening JSON file
f = open('unmask.json')
dirty_map = json.load(f)

with open(dir_csv) as file:
    content = file.readlines()

for i in content:
    index = content.index(i)
    lyric = i.split(',')[1]
    if lyric.__contains__('*'):
      dirty_words = extract_and_clean_words(lyric)
      for j in dirty_words:
        if j not in dirty_map.keys():
          print(lyric)
          print(f'error locating {j} in the map')
        else:
          content[index] = content[index].replace(j,dirty_map[j])
          print(f'{j} replaced by {dirty_map[j]}')

# Closing file
f.close()

In [ ]:
for i in content:
      row_to_write = i.strip().split(',')
      # Ensure the newline character is removed from the end of the second element
      row_to_write = [row_to_write[0], row_to_write[1].replace('\n', '')]
      # print(row_to_write)
      # Write the split elements into the CSV file
      append_rows_to_csv('metadata8.0.csv', [row_to_write])

In [93]:
# @title check number of chunks in total
import os
n = 0
for i in os.listdir('/content/drive/MyDrive/data/dataset_cleaned/train'):
  n = n+1
training = n
print(f"number of training data: {n}")
for i in os.listdir('/content/drive/MyDrive/data/dataset_cleaned/test'):
  n = n+1
print(f"number of testing data: {n-training}")
print(f"number of data: {n}")
n_chunks = n

number of training data: 12553
number of testing data: 3154
number of data: 15707
